# Quantum Annealer for Philogenetic Trees

---

In this notebook, we are going to use D-Wave's Ocean to create the needed optimization problem to reconstruct Philogenetic Trees. In short, this is where the real work begins. This work will be based in 2 documents, a book that describes Quantum Annealing [1], and the paper for reconstruction of Philogenetic Trees [2].



[1] Combarro, E. F., & Gonzalez-Castillo, S. (2023). A practical guide to quantum machine learning and quantum optimisation: Hands-On Approach to Modern Quantum Algorithms. Packt Publishing.

[2] Onodera, W., Hara, N., Aoki, S., Asahi, T., & Sawamura, N. (2022). Phylogenetic tree reconstruction via graph cut presented using a quantum-inspired computer. Molecular Phylogenetics and Evolution, 178, 107636. https://doi.org/10.1016/j.ympev.2022.107636

In [1]:
import numpy as np
import dimod
from dimod import BinaryQuadraticModel, BINARY
from typing import Optional
from dwave.system import DWaveSampler, EmbeddingComposite
from colorama import Fore

First, we start with an example:

In [46]:
# Coefficients of the quadratic term elements (squared or products)
J = {(0,1):1, (0,2):1}
# Coefficients of the linear terms
h = {}
problem = BinaryQuadraticModel(h, J, 0.0, BINARY)
print("The problem we are going to solve is:")
print(problem)

The problem we are going to solve is:
BinaryQuadraticModel({0: 0.0, 1: 0.0, 2: 0.0}, {(1, 0): 1.0, (2, 0): 1.0}, 0.0, 'BINARY')


From the paper we have that the minimization is defined as:

$$Min_{cut}=\sum_{i=1}^{n-1}\sum_{j=i+1}^n d_{ij}(x_i-x_j)^2,\qquad x_i=\{0,1\}, \quad i = 1,...,n.$$

Where $d_{ij}$ is the element $ij$ from the matrix $D$, where the differences between elements are represented. In other words, if you consider the problem as a graph, $D$ is the adjacency matrix from the graph.

If we take a closer look at this formula, we can see that we take the top part of the matrix, also, we start counting from 1, and I don't want that, so we can rewrite the expression as:

$$Min_{cut}=\sum_{i=0}^{n-2}\sum_{j=i+1}^{n-1} d_{ij}(x_i-x_j)^2,\qquad x_i=\{0,1\}, \quad i = 0,...,n-1.$$

To start, as I don't have any data to create adjacency matrices, I can create a random all-to-all graph with the characteristics defined in the paper. The graph would be the next:

<div style="text-align: center;">
    <img src="images/randgraph_1.png" alt="Complete Graph" width="600px">
</div>

Defined by the following adjacency matrix:

$$
\begin{pmatrix}
    0  & 92 & 73 & 78 & 92 \\
    92 & 0  & 21 & 49 & 34 \\
    73 & 21 & 0  & 35 & 63 \\
    78 & 49 & 35 & 0  & 29 \\
    92 & 34 & 63 & 29 & 0 \\
\end{pmatrix}
$$

Firstly, we can create a function that creates ``BinaryQuadraticModel`` objects from a given matrix for our problem. This will facilitate the process hereafter.

In [20]:
# Function to create BinaryQuadraticModel from a numpy matrix
def create_problem (matrix:np.ndarray)->BinaryQuadraticModel:
    r"""
    Creates a BinaryQuadraticModel from a numpy matrix using the Min-cut formulation. Both simmetrical matrices and matrices with 0 above the main diagonal work.
    
    Args:
        `matrix`: Matrix that defines the problem.
    
    Returns:
        The BinaryQuadraticModel from dimod that defines the problem.
    """
    
    rows = matrix.shape[0]
    var = []
    
    for i in range(rows):
        var.append(dimod.Binary('x'+str(i)))
        
    obj = BinaryQuadraticModel({},{},0.0,BINARY)
    
    for i in range(rows):
        for j in range(i):
            obj+=matrix[i,j]*(var[i]-var[j])**2
            
    return obj

In [21]:
matrix = np.array([[0,0,0,0,0],[92,0,0,0,0],[73,21,0,0,0],[78,49,35,0,0],[92,34,63,29,0]])

problem = create_problem(matrix)
print(problem)

BinaryQuadraticModel({'x1': 196.0, 'x0': 335.0, 'x2': 192.0, 'x3': 191.0, 'x4': 218.0}, {('x0', 'x1'): -184.0, ('x2', 'x1'): -42.0, ('x2', 'x0'): -146.0, ('x3', 'x1'): -98.0, ('x3', 'x0'): -156.0, ('x3', 'x2'): -70.0, ('x4', 'x1'): -68.0, ('x4', 'x0'): -184.0, ('x4', 'x2'): -126.0, ('x4', 'x3'): -58.0}, 0.0, 'BINARY')


It's important to know that the next cell access the D-Wave Quantum Annealer. That's why there's commented lines.

In [ ]:
# You need to have an access token configured
# sampler = EmbeddingComposite(DWaveSampler())
# result = sampler.sample(problem, num_reads=10)
print("The solutions that we have obtained are")
print(result)

The solutions that we have obtained are
   0  1  2  3  4 energy num_oc. chain_.
0  0  0  0  0  0    0.0       5     0.0
1  1  1  1  1  1    0.0       5     0.0
['BINARY', 2 rows, 10 samples, 5 variables]


In [55]:
print(f'Time required to complete: {result.info['timing']['qpu_access_time']}ms')

Time required to complete: 16541.56ms


In [45]:
x0 = dimod.Binary('x0')
x1 = dimod.Binary('x1')
x2 = dimod.Binary('x2')

blp = dimod.ConstrainedQuadraticModel()

blp.set_objective(3*(x0-x1)**2+3*(x1-x2)**2+(x0-x2)**2+24*(x0+x1+x2-1)**2)
# blp.add_constraint(x0+x1+x2 == 1 )
# blp.add_constraint(x0+x1+x2 <= 2 )

print(blp)

solver = dimod.ExactCQMSolver()

sol = solver.sample_cqm(blp)

print('Solutions')
print(sol,'\n')

# We want the best feasible solution. We can filter by its feasibility and take the first element
feas_sol = sol.filter(lambda s: s.is_feasible)
print(Fore.RED+'Best Solution'+Fore.RESET)
print(f'Variables: {feas_sol.first.sample}, Cost = {feas_sol.first.energy}')

Constrained quadratic model: 3 variables, 0 constraints, 6 biases

Objective
  24 - 20*Binary('x0') - 18*Binary('x1') - 20*Binary('x2') + 42*Binary('x0')*Binary('x1') + 46*Binary('x0')*Binary('x2') + 42*Binary('x1')*Binary('x2')

Constraints

Bounds

Solutions
  x0 x1 x2 energy num_oc. is_sat. is_fea.
2  1  0  0    4.0       1 arra... np.T...
4  0  0  1    4.0       1 arra... np.T...
1  0  1  0    6.0       1 arra... np.T...
0  0  0  0   24.0       1 arra... np.T...
3  1  1  0   28.0       1 arra... np.T...
5  0  1  1   28.0       1 arra... np.T...
6  1  0  1   30.0       1 arra... np.T...
7  1  1  1   96.0       1 arra... np.T...
['INTEGER', 8 rows, 8 samples, 3 variables] 

Best Solution
Variables: {'x0': np.int64(1), 'x1': np.int64(0), 'x2': np.int64(0)}, Cost = 4.0


In [144]:
qubo = dimod.cqm_to_bqm(blp,lagrange_multiplier=5)
print(qubo)

(BinaryQuadraticModel({'x0': 3.0, 'x1': 5.0, 'x2': 3.0}, {('x1', 'x0'): -4.0, ('x2', 'x0'): 0.0, ('x2', 'x1'): -4.0}, 1.0, 'BINARY'), <dimod.constrained.constrained.CQMToBQMInverter object at 0x000002089A377640>)


In [48]:
# Function to create BinaryQuadraticModel from a numpy matrix
def create_cqm_problem (matrix:np.ndarray,c=0,alpha=0)->dimod.ConstrainedQuadraticModel:
    r"""
    Creates a ConstrainedQuadraticModel from a numpy matrix using the Min-cut formulation. Both simmetrical matrices and matrices with 0 above the main diagonal work.
    
    Args:
        `matrix`: Matrix that defines the problem.
        `c`: Number of non zero results wanted.
        `alpha`: Factor to amplify. Higher alpha (>1) achieve the number of `c` of non zero results easily. If 0, no defined `c` is set, thus 11...11 and 00...00 become the best solution.
    
    Returns:
        The ConstrainedQuadraticModel from dimod that defines the problem.
    """
    
    rows = matrix.shape[0]
    var = []
    
    for i in range(rows):
        var.append(dimod.Binary('x'+str(i)))
        
    obj = BinaryQuadraticModel({},{},0.0,BINARY)
    
    for i in range(rows):
        for j in range(i):
            obj+=matrix[i,j]*(var[i]-var[j])**2
    
    suma = np.sum(var)      
    # Add restriction term
    obj+=alpha*(suma-c)**2
    
    problem = dimod.ConstrainedQuadraticModel()
    
    problem.set_objective(obj)
    
    return problem

In [43]:
matrix = np.array([[0,0,0,0,0],[92,0,0,0,0],[73,21,0,0,0],[78,49,35,0,0],[92,34,63,29,0]])

problem = create_cqm_problem(matrix,c=2,alpha=100)

sol = solver.sample_cqm(problem)

print(problem)

print('Solutions')
print(sol,'\n')

# We want the best feasible solution. We can filter by its feasibility and take the first element
feas_sol = sol.filter(lambda s: s.is_feasible)
print(Fore.RED+'Best Solution'+Fore.RESET)
print(f'Variables: {feas_sol.first.sample}, Cost = {feas_sol.first.energy}')

Constrained quadratic model: 5 variables, 0 constraints, 15 biases

Objective
  400 - 104*Binary('x1') + 35*Binary('x0') - 108*Binary('x2') - 109*Binary('x3') - 82*Binary('x4') + 16*Binary('x1')*Binary('x0') + 158*Binary('x1')*Binary('x2') + 54*Binary('x0')*Binary('x2') + 102*Binary('x1')*Binary('x3') + 44*Binary('x0')*Binary('x3') + 130*Binary('x2')*Binary('x3') + 132*Binary('x1')*Binary('x4') + 16*Binary('x0')*Binary('x4') + 74*Binary('x2')*Binary('x4') + 142*Binary('x3')*Binary('x4')

Constraints

Bounds

Solutions
   x0 x1 x2 x3 x4 energy num_oc. is_sat. is_fea.
20  0  0  1  0  1  284.0       1 arra... np.T...
10  0  1  0  1  0  289.0       1 arra... np.T...
8   0  0  0  1  0  291.0       1 arra... np.T...
4   0  0  1  0  0  292.0       1 arra... np.T...
2   0  1  0  0  0  296.0       1 arra... np.T...
12  0  0  1  1  0  313.0       1 arra... np.T...
16  0  0  0  0  1  318.0       1 arra... np.T...
6   0  1  1  0  0  346.0       1 arra... np.T...
18  0  1  0  0  1  346.0       1 ar

In [40]:
qprob = dimod.cqm_to_bqm(problem)

print(qprob)

(BinaryQuadraticModel({'x1': 196.0, 'x0': 335.0, 'x2': 192.0, 'x3': 191.0, 'x4': 218.0}, {('x0', 'x1'): -184.0, ('x2', 'x1'): -42.0, ('x2', 'x0'): -146.0, ('x3', 'x1'): -98.0, ('x3', 'x0'): -156.0, ('x3', 'x2'): -70.0, ('x4', 'x1'): -68.0, ('x4', 'x0'): -184.0, ('x4', 'x2'): -126.0, ('x4', 'x3'): -58.0}, 0.0, 'BINARY'), <dimod.constrained.constrained.CQMToBQMInverter object at 0x00000233FF201570>)


After this tests, we see that with a high enough $\alpha$ we can guarantee that the best solution contais the subdivision that we want. Knowing this, with good management of the variables we can achieve a solution without adding consraints. Thus, we will refrain from using constraints and use directly the `BinaryQuadraticModel`. However, we will mantain the function `create_cqm_problem` for future convenience of obtaining all solutions in a simple way.

In the next cell, there will be the final problem creation function, so **refrain from using the function from cell 8**.

In [50]:
def min_cut_c (matrix:np.ndarray,c=0,alpha=0)->BinaryQuadraticModel:
    r"""
    Creates a BinaryQuadraticModel from a numpy matrix using the Min-cut formulation. Both simmetrical matrices and matrices with 0 above the main diagonal work.
    
    Args:
        `matrix`: Matrix that defines the problem.
        `c`: Number of non zero results desired.
        `alpha`: Factor to amplify. Higher alpha (>1) achieve the number of `c` of non zero results easily. If 0, no defined `c` is set, thus 11...11 and 00...00 become the best solution.
    
    Returns:
        The BinaryQuadraticModel from dimod that defines the problem.
    """
    
    rows = matrix.shape[0]
    var = []
    
    for i in range(rows):
        var.append(dimod.Binary('x'+str(i)))
        
    obj = BinaryQuadraticModel({},{},0.0,BINARY)
    
    for i in range(rows):
        for j in range(i):
            obj+=matrix[i,j]*(var[i]-var[j])**2
    
    suma = np.sum(var)      
    # Add restriction term
    obj+=alpha*(suma-c)**2
    
    return obj

In [105]:
# Test the code using D-Wave QA
matrix = np.array([[0,0,0,0,0],[92,0,0,0,0],[73,21,0,0,0],[78,49,35,0,0],[92,34,63,29,0]])

problem = min_cut_c(matrix,c=2,alpha=200)

print(problem)

sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(problem, num_reads=10)
print("The solutions that we have obtained are")
print(result)

BinaryQuadraticModel({'x1': -404.0, 'x0': -265.0, 'x2': -408.0, 'x3': -409.0, 'x4': -382.0}, {('x0', 'x1'): 216.0, ('x2', 'x1'): 358.0, ('x2', 'x0'): 254.0, ('x3', 'x1'): 302.0, ('x3', 'x0'): 244.0, ('x3', 'x2'): 330.0, ('x4', 'x1'): 332.0, ('x4', 'x0'): 216.0, ('x4', 'x2'): 274.0, ('x4', 'x3'): 342.0}, 800.0, 'BINARY')
The solutions that we have obtained are
  x0 x1 x2 x3 x4 energy num_oc. chain_.
0  0  0  1  0  1  284.0       2     0.0
1  0  1  0  1  0  289.0       6     0.0
2  0  0  1  1  0  313.0       1     0.0
3  0  1  0  0  1  346.0       1     0.0
['BINARY', 4 rows, 10 samples, 5 variables]


In [106]:
print(f'Time required to complete: {result.info['timing']['qpu_access_time']}us')
print(f'Best solution energy: {result.first.energy}')
print(f'Best solution configuration: {result.first.sample}')

n_graph_0 = [i for i in result.first.sample if result.first.sample[i]==0]
n_graph_1 = [i for i in result.first.sample if result.first.sample[i]==1]

print(f'The cut graph are:\nGraph 1: {n_graph_0}\nGraph 2: {n_graph_1}')

Time required to complete: 16919.56us
Best solution energy: 284.0
Best solution configuration: {'x0': np.int8(0), 'x1': np.int8(0), 'x2': np.int8(1), 'x3': np.int8(0), 'x4': np.int8(1)}
The cut graph are:
Graph 1: ['x0', 'x1', 'x3']
Graph 2: ['x2', 'x4']


In [107]:
indices = [int(i[1]) for i in n_graph_0]
indices2 = [int(i[1]) for i in n_graph_1]

new_matrix = matrix[np.ix_(indices, indices)]

print(new_matrix)


[[ 0  0  0]
 [92  0  0]
 [78 49  0]]


In [103]:
def n_cut(score,ng0,ng1,og):
    r"""
    Returns the Ncut from a cut.
    
    Args:
        `score`: The energy level obtained from the Mincut method.
        `ng0`: The nodes defining one of the new graphs.
        `ng1`: The nodes defining the other new graph.
        `og`: The original matrix.
    Returns:
        Ncut energy level.
    """
    asoc0 = np.sum(og[ng0, :])
    asoc1 = np.sum(og[ng1, :])    
    
    ncut = (score/asoc0) + (score/asoc1)
    
    return ncut

In [108]:
n_cut(result.first.energy,indices,indices2,matrix)

np.float64(2.028366646476883)

After this, we can define the full algorithm, without the initial matrix definition.

In [ ]:
def philo_tree(matrix:np.ndarray):
    r"""
    Recursive function that uses D-Wave QA to create the tree using Ncut
    
    Args:
        `matrix`: The matrix defining the graph
        Probably you could define alpha from outside.
    """
    
    rows = matrix.shape[0]
    
    var = int(np.floor(rows/2.0))+1
    ncuts = []
    n_graph_0 = []
    n_graph_1 = []
    sampler = EmbeddingComposite(DWaveSampler())

    # Run min_cut for each configuration
    for i in range(1,var):
        problem = min_cut_c(matrix,c=i,alpha=200)
        result = sampler.sample(problem, num_reads=10)
        n_graph_0.append([int(i[1]) for i in result.first.sample if result.first.sample[i]==0])
        n_graph_1.append([int(i[1]) for i in result.first.sample if result.first.sample[i]==1])
        ncuts.append(n_cut(result.first.energy,n_graph_0,n_graph_1,matrix))
    
    index = np.argmin(ncuts)
    
    # Recursivity in the first graph
    if len(n_graph_0[index]) > 2:
        nmatrix1 = matrix[np.ix_(n_graph_0[index], n_graph_0[index])]
        philo_tree(nmatrix1)
        
    # Recursivity in the first graph
    if len(n_graph_1[index]) > 2:   
        nmatrix2 = matrix[np.ix_(n_graph_1[index], n_graph_1[index])]
        philo_tree(nmatrix2)
        

In [ ]:
ncuts = [2,1,3,4]
n_graph_0 = []

n_graph_0.append([1,2,3])
n_graph_0.append([4,5,6])
len(n_graph_0[np.argmin(ncuts)])

SyntaxError: invalid syntax (4182533435.py, line 12)

### TODO MAÑANA

- [ ] Ver que cojones devolver
- [ ] Hacer pruebas